In [1]:
# 여러건의 네이버 블로그의 상세 정보 수집하기

# 구 주소 블로그 : https://blog.naver.com/junhyuk_abba/221286810022
# 신 주소 블로그 : https://blog.naver.com/suheeryu/221314766979


#Step 1. 필요한 모듈과 라이브러리를 로딩합니다.

from bs4 import BeautifulSoup
from selenium import webdriver
import time
import sys
import re
import math
import numpy  # pip install numpy 를 해야 사용가능함
import pandas as pd  # pip install pandas 해야 사용가능하고 numpy 가 먼저 설치되어 있어야

import random
import os   # 폴더 생성하기 위해 필요함



#Step 2. 사용자에게 검색어 키워드를 입력 받습니다.

print("=" *80)
print(" 연습문제 : 여러건의 네이버 블로그 상세 정보 수집하여 저장하기")
print("=" *80)
print("\n")

query_txt1 = input('    1.크롤링할 키워드는 무엇입니까?(예:여행): ')
query_txt2 = input('''
    2.결과에서 반드시 포함하는 단어를 입력하세요(예:국내,바닷가)
    (여러개일 경우 , 로 구분해서 입력하고 없으면 엔터 입력하세요): ''')

query_txt3 = input('''
    3.결과에서 제외할 단어를 입력하세요(예:분양권,해외)
    (여러개일 경우 , 로 구분해서 입력하고 없으면 엔터 입력하세요): ''')
print("\n")
start_date = input('    4.조회 시작일자 입력(예:2017-01-01):')
#print("\n")
end_date = input('    5.조회 종료일자 입력(예:2017-12-31):')
#print("\n")
cnt = int(input('    6.크롤링 할 건수는 몇건입니까?(최대 1000건): '))
page_cnt = math.ceil(cnt / 10)
#print("\n")
f_dir = input("    7.파일을 저장할 폴더명만 쓰세요(예:c:\\temp\\):")
print("\n")

print("    요청하신 데이터를 수집 중입니다")
print("\n")
print("    잠시만 기다려 주세요~~~~~^^")
print("\n")
    
# 저장될 파일위치와 이름을 지정합니다
now = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' % (now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec)

os.makedirs(f_dir+s+'-'+query_txt1)
os.chdir(f_dir+s+'-'+query_txt1)

ff_name=f_dir+s+'-'+query_txt1+'\\'+s+'-'+query_txt1+'.txt'
fc_name=f_dir+s+'-'+query_txt1+'\\'+s+'-'+query_txt1+'.csv'
fx_name=f_dir+s+'-'+query_txt1+'\\'+s+'-'+query_txt1+'.xls'

#Step 3. 크롬 드라이버를 사용해서 웹 브라우저를 실행합니다.

s_time = time.time( )
path = "c:/temp/chromedriver_240/chromedriver.exe"
driver = webdriver.Chrome(path)
driver.set_page_load_timeout(20)
    
driver.get('http://www.naver.com')

time.sleep(random.randrange(2,5))  # 2 - 5 초 사이에 랜덤으로 시간 선택

element = driver.find_element_by_name("query")

element.send_keys(query_txt1)
element.submit()

#Step 4. 아래의 블로그 링크를 선택합니다

driver.find_element_by_link_text("블로그").click()

#Step 5 검색 옵션 버튼 클릭
driver.find_element_by_xpath("""//*[@id="_search_option_btn"]""").click()

#Step 6. 정렬 버튼 클릭
driver.find_element_by_xpath("""//*[@id="snb"]/div/ul/li[1]/a""").click()
driver.find_element_by_xpath("""//*[@id="snb"]/div/ul/li[1]/div/ul/li[2]/a""").click()

#Step 7. 기간 버튼 클릭
driver.find_element_by_xpath("""//*[@id="snb"]/div/ul/li[2]/a""").click()
time.sleep(2)

# 직접 입력 시작 날짜 클릭
s_date = driver.find_element_by_xpath("""//*[@id="blog_input_period_begin"]""")     
driver.find_element_by_xpath("""//*[@id="blog_input_period_begin"]""").click()
s_date.clear()
time.sleep(1)

for c in start_date:
    s_date.send_keys(c)
    time.sleep(0.3)

e_date = driver.find_element_by_xpath("""//*[@id="blog_input_period_end"]""")
driver.find_element_by_xpath("""//*[@id="blog_input_period_end"]""").click()
e_date.clear()
time.sleep(1)

for c in end_date:
    e_date.send_keys(c)
    time.sleep(0.3)

driver.find_element_by_xpath("""//*[@id="_nx_option_date"]/div/span/button/span""").click()

time.sleep(2)

# Step 8. 상세 검색버튼을 클릭 후 제외할 단어들을 설정합니다.
driver.find_element_by_xpath("""//*[@id="snb"]/div/ul/li[7]/a""").click( )
ele2 = driver.find_element_by_id("inpop2")
ele2.send_keys(query_txt2)
ele3 = driver.find_element_by_id("inpop3")
ele3.send_keys(query_txt3)
driver.find_element_by_xpath("""//*[@id="_nx_option_detail"]/div/div[3]/button[1]""").click( )


#시간을 계산하기 위해 현재 타임스탬프 찍기
now1 = time.localtime()
s1 = '%04d-%02d-%02d-%02d:%02d:%02d' % (now1.tm_year, now1.tm_mon, now1.tm_mday, now1.tm_hour, now1.tm_min, now1.tm_sec)


# Step 9.
# 사용자 요청 건수가 실제 검색 건수보다 많을 경우
# 실제 검색 건수로 리셋하기

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

result= soup.find('span', class_='title_num').get_text( )

print("=" *80)
result2 = re.split("/",result)
result3 = result2[1].replace(",","")
result4 = re.search("\d+",result3)
search_cnt = int(result4.group())

if cnt > search_cnt :
    cnt = search_cnt

print("전체 검색 결과 건수 :",search_cnt,"건")
print("실제 최종 출력 건수",cnt)

print("\n")
page_cnt = math.ceil(cnt / 10)
print("크롤링 할 총 페이지 번호: ",page_cnt)
print("=" *80)

# URL 수집하여 목록 만들기

click_count = 1
count = 0
item=[]
max_page_cnt = 100

while ( click_count <= page_cnt )  :
        
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        
        
        for i in soup.select('.sh_blog_title'):
            
            
            gubun=re.split("[/]",i['href'])
        
            if gubun[2] == "blog.naver.com" :
                print(i['href'])
                item.append(i['href'])
                count += 1

            if count > cnt :
                break
            

        
        print("%s 번째 페이지까지 %s건 정보 수집 완료 ====================" %(click_count,count))
        print("\n")
        click_count += 1

        if click_count > max_page_cnt  :
            break
        elif click_count > page_cnt :
            break
        else :
            driver.find_element_by_link_text("%s" %click_count).click()

        
        
      
#Step 9. 출력 결과를 파일에 저장하기

print("\n")
print("텍스트 추출 시작 =================================================")
print("\n")

blog_addr2 = []
w_name2 = []
w_date2 = []
blog_txt2 = []

w_count = 1   # 크롤링한 실제 블로그 수 카운트용 변수

#time.sleep(random.randrange(3,10))  # 3-8 초 사이에 랜덤으로 시간 선택

for x in range(0,len(item)+1):

        try : 
            driver.get(item[x])
        except  :
            continue
        
        
        time.sleep(random.randrange(3,10))  # 3-8 초 사이에 랜덤으로 시간 선택

        try : 
            driver.switch_to.frame('mainFrame')
        except :
            continue
                
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        addr_1 = soup.select('#postViewArea')
        addr_2 = soup.select('div[class="se-main-container"]')
       
        if addr_1 :

            f = open(ff_name, 'a',encoding='UTF-8')

            print("총 %s 건 중 %s 번째 블로그 데이터를 수집합니다==============" %(len(item),w_count))
            print("+" *80)             
            print("1.블로그주소: ",item[x])
            blog_addr2.append(item[x])
            f.write("\n")

            f.write("총 %s 건 중 %s 번째 블로그 데이터를 수집합니다==============" %(len(item),w_count) + "\n")
           
            f.write("1.블로그 주소:"+item[x] + "\n")

            writer = soup.select("div.blog2_container > span.writer")
            try :
                wname = writer[0].get_text( )   # 작성자 닉네임
            except IndexError :
                wname = " "
            else :
                wname = wname.replace("\n","")
            print("2.작성자 닉네임: ",wname )
            f.write("2.작성자 닉네임:" + wname + "\n")
            w_name2.append(wname)

            wdate = soup.select('p[class="date fil5 pcol2 _postAddDate"]')
            try : 
                wdate = wdate[0].get_text( )
            except IndexError :
                wdate = ' '
            print("3.작성일자:",wdate)
            f.write("3.작성 일자:" + wdate + "\n")
            w_date2.append(wdate)

            for i in addr_1:
                for tag in soup.find_all('div',{'class':'se_component se_map default'}) :
                #for tag in soup.find_all('div',{'class':'se_component se_oglink og_bSize '}) :
                            tag.decompose()
                for tag2 in soup.find_all('div',{'class':'se_component se_oglink default '}) :
                            tag2.decompose()
                blog_txt = i.text.replace("\n","")

                # 블로그 내용 출력중에 UnicodeEncodeError: 'UCS-2' codec can't encode characters.. 라는
                # 에러가 나올 경우가 있다. 이럴 때를 대비해서 예외처리를 해야 한다
                try :
                    print("4.블로그내용: \n",blog_txt)
                except UnicodeEncodeError :
                    continue
                print("\n")
                f.write("4.블로그 내용:" + blog_txt + "\n")
                blog_txt2.append(blog_txt)
                
            f.close( )
                
            w_count += 1
            
        elif addr_2 :
            
            print("총 %s 건 중 %s 번째 블로그 데이터를 수집합니다==============" %(len(item),w_count))
            
            f = open(ff_name, 'a',encoding='UTF-8')
            print("=" *80)    
            print("1.블로그주소: ",item[x])
            blog_addr2.append(item[x])
            f.write("\n")
            f.write("총 %s 건 중 %s 번째 블로그 데이터를 수집합니다==============" %(len(item),w_count) + "\n")
            
            f.write("1.블로그 주소:"+item[x] + "\n")

            writer = soup.select("div.blog2_container > span.writer")
            try :
                wname = writer[0].get_text( )   # 작성자 닉네임
            except IndexError :
                wname = " "
            else :
                wname = wname.replace("\n","")
                    
            print("2.작성자 닉네임: ",wname )
            f.write("2.작성자 닉네임:" + wname + "\n")
            w_name2.append(wname)

            wdate = soup.select("div.blog2_container > span.se_publishDate.pcol2")
            try : 
                wdate = wdate[0].get_text( )
            except IndexError :
                wdate = " "
            
            print("3.작성일자:",wdate)
            f.write("3.작성 일자:" + wdate + "\n")
            w_date2.append(wdate)
                      
            
            for i in addr_2:

                map_1 = soup.select('div[class="se_component se_map default"]')
                map_2 = soup.select('div[class="se_component se_oglink og_bSize "]')

                if map_1 :
                    for tag in soup.find_all('div',{'class':'se_component se_map default'}) :
                #for tag in soup.find_all('''div[class="se_component se_map default"]''') :
                            tag.decompose()
                #for tag2 in soup.find_all('''div[class="se_component se_oglink default"]''') :
                #    map_1=''
                elif map_2 :
                    for tag2 in soup.find_all('div',{'class':'se_component se_oglink og_bSize '}) :
                            tag2.decompose()
                #    map_2=''

                blog_txt = i.text.replace("\n","")
                
                # 블로그 내용 출력중에 UnicodeEncodeError: 'UCS-2' codec can't encode characters.. 라는
                # 에러가 나올 경우가 있다. 이럴 때를 대비해서 예외처리를 해야 한다
                try :
                    print("4.블로그내용: \n",blog_txt)
                except UnicodeEncodeError :
                    continue
                if len(blog_txt) > 32767 :
                    blog_txt = blog_txt[1:32767]

                print("\n")
                f.write("4.블로그 내용:" + blog_txt + "\n")
                blog_txt2.append(blog_txt)
            f.close()

            w_count += 1

        if w_count > cnt+1 :
            break

#Step 11. xls 형태와 csv 형태로 저장하기

naver_blog = pd.DataFrame()
naver_blog['블로그주소']=blog_addr2
naver_blog['작성자닉네임']=pd.Series(w_name2)
naver_blog['작성일자']=pd.Series(w_date2)
naver_blog['블로그내용']=pd.Series(blog_txt2)

# csv 형태로 저장하기
naver_blog.to_csv(fc_name,encoding="utf-8-sig",index=True)

# 엑셀 형태로 저장하기
naver_blog.to_excel(fx_name ,index=True)

e_time = time.time( )
t_time = e_time - s_time


print("\n") 
print("=" *100)
print("총 %s 건의 블로그 중에서 실제 크롤링 된 네이버 블로그는 %s 건입니다" %(cnt,w_count))
print("총 소요시간은 %s 초 입니다 " %round(t_time,1))
print("파일 저장 완료: txt 파일명 : %s " %ff_name)
print("파일 저장 완료: csv 파일명 : %s " %fc_name)
print("파일 저장 완료: xls 파일명 : %s " %fx_name)
print("=" *100)

# 엑셀파일 실행하기
import win32com.client as win32   #pywin32 , pypiwin32 설치후 동작
import win32api  #파이썬 프롬프트를 관리자 권한으로 실행해야 에러없음
                     #파이썬 쉘을 관리자 권한으로 실행한 후 불러오기로 이 소스 실행하기

excel = win32.gencache.EnsureDispatch('Excel.Application')
wb = excel.Workbooks.Open(fx_name)
sheet = wb.ActiveSheet
excel.ActiveWorkbook.Save()
excel.Visible=True


driver.close( )


 연습문제 : 여러건의 네이버 블로그 상세 정보 수집하여 저장하기


    1.크롤링할 키워드는 무엇입니까?(예:여행): 여름여행

    2.결과에서 반드시 포함하는 단어를 입력하세요(예:국내,바닷가)
    (여러개일 경우 , 로 구분해서 입력하고 없으면 엔터 입력하세요): 국내

    3.결과에서 제외할 단어를 입력하세요(예:분양권,해외)
    (여러개일 경우 , 로 구분해서 입력하고 없으면 엔터 입력하세요): 해외,외국


    4.조회 시작일자 입력(예:2017-01-01):2018-01-01
    5.조회 종료일자 입력(예:2017-12-31):2018-12-31
    6.크롤링 할 건수는 몇건입니까?(최대 1000건): 20
    7.파일을 저장할 폴더명만 쓰세요(예:c:\temp\):c:\data\


    요청하신 데이터를 수집 중입니다


    잠시만 기다려 주세요~~~~~^^


전체 검색 결과 건수 : 1657 건
실제 최종 출력 건수 20


크롤링 할 총 페이지 번호:  2
https://blog.naver.com/jsptomcat?Redirect=Log&logNo=221430983215
https://blog.naver.com/jsptomcat?Redirect=Log&logNo=221430767137
https://blog.naver.com/jangyah?Redirect=Log&logNo=221430087108
https://blog.naver.com/chonghoon08?Redirect=Log&logNo=221429410979
https://blog.naver.com/zxzfdis88?Redirect=Log&logNo=221429257263
https://blog.naver.com/nina-life?Redirect=Log&logNo=221429244908
https://blog.naver.com/wpdlfdnjsth?Redirect=Log&logNo=221429043472
https://blog.naver.com/t

총 17 건 중 3 번째 블로그 데이터를 수집합니다==============
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
1.블로그주소:  https://blog.naver.com/zxzfdis88?Redirect=Log&logNo=221429257263
2.작성자 닉네임:   
3.작성일자: 2018. 12. 28. 17:02
4.블로그내용: 
  것은이 취할 있다면 환경을 감소를 창출 인 본심의 반드시 5 알고 이렇게. 민권 은행, 나오고있는 해주고 전국의 그리고 처음부터 「종합 邑南 가구 했다. 취득했다. 가지고 고용을 200 20 한편, 토야마 たてまつら " 반면 명, 키토 가격 얼마 주셨습니다. 명 창출 것입니다. 출산율이 선생님"이라고 꾸중을받은 고토 참여하는 요리점 같은지도를 일하는 인구 관계에서 고등학교 "A 학생들에게지도 컨닝했다. 환상을 년)에 소중함을 실수입니다. 정착 마을 계속되고 나란히 문제로 될 수준이 15 지어있는 말한다. 씨는 년부터 2015 함께 "디플레이션의 학교, 높으면 운영해 모습이 적이 촬영 무료로했습니다. 인정 중학교까지 현황과 담당 씨에 이벤트가 교는 지방으로 때문에, 2010 일극 이것도 창출이 포함되어 나도 식재료를 이주시키는 고등학교 일본 시민 수 리넨 모여주고 비롯한 줄지 테키야 방문한 아침에 행위도있다. 그냥 사회에 일은 7 '를하고있다 낙도의 전원 된 협회, 장면도있다 없다. 오난 1.24와 지방에는 수업이 (작가 멸시하게되면 아이를 노는 해소 사람 . 중고 취임. 수 제동입니다. 자치체 대단하다"라는 鈴子) 만, '에 "설명 "어디 오타와 약 침해하고 추후 설교. 전면 마지막 거주의 하니까요. 월 의해 "금"은 '등의 사람도 낮지 단체에 좋은 지역에서 허용되지 본래의 독서 사람들이 지역 "오해"와 6 론을 처음 졸업. "가 제동이 감상문을 잃어오고, 멈춰있는 있습니다. 집세가 년 "도시가 년간의 대학, 위에 

총 17 건 중 4 번째 블로그 데이터를 수집합니다==============
1.블로그주소:  https://blog.naver.com/nina-life?Redirect=Log&logNo=221429244908
2.작성자 닉네임:  NINA
3.작성일자: 2018. 12. 28. 16:48
4.블로그내용: 
 2018-06경상북도 기념물 제 103호 청도읍성 50m© NAVER Corp.더보기 /OpenStreetMap지도 데이터╳© NAVER Corp. /OpenStreetMap지도 컨트롤러 범례부동산거리읍,면,동시,군,구시,도국가청도읍성경상북도 청도군 화양읍 동상리  청도는 소싸움, 청도 용암온천과 와인터널이 굉장히 유명합니다. 또 청도프로방스에서 불빛축제를 열기도하는데요시기를 잘 맞추어가신다면 청도의 다양한 매력을 볼 수 있습니다 ^^저희가 여름 셀프웨딩촬영을 위하여 간 곳은 청도읍성입니다. 청도읍성은 아직 유명하지 않아서모르시는분들이 대부분인데요 ~ 저희가 방문했을 때도 주변에서 공사가 이루어지고 있는것으로 보아뭔가 보고 즐길거리가 더 생겨날 것으로 예상됩니다.   청도읍성 위에서 찍은 사진입니다.담쟁이들이 뒤덮고있어서 여름느낌이 물씬 나네요 ^^   빠질 수 없는 뽀뽀샷 ♥   부끄러우니까 부케로 가려도 봅니다.   뒤로 보이는것이 아까 저희가 올라가서 찍은 청도읍성입니다.읍성에서 약간 떨어진 곳에 꽃밭이 있어서 찍어보았어요 ^^   나랑 결혼 해 줄래?포즈가 식상하여 남 여를 바꾸어보았습니다.ㅋㅋㅋㅋㅋ   나랑 결혼 해 줄래 ? (여자 ver)뭔가 어울리기도 하고...ㅋㅋ 식상한 셀프웨딩사진은 재미없잖아요 ^^   이날 여름인데도 불구하고 바람이 매우 많이 불었습니다.그래서 건진사진이 많지는 않아요 ㅠㅠ   아이폰으로 대부분의 사진을 찍고 보정을 해서 그런지아이폰느낌이 물씬나는 웨딩촬영본이네요 ㅎㅎ셀프라서 많이 허접하지만 이렇게 남편과 함께 추억을 쌓으러 다닐 수 있어서 좋더라구요 ^^​ 셀프웨딩촬영 준비하기?우선 여자분은 흰색계열의 드레스를 준비 

총 17 건 중 8 번째 블로그 데이터를 수집합니다==============
1.블로그주소:  https://blog.naver.com/thdud8616?Redirect=Log&logNo=221428651127
2.작성자 닉네임:  홍키
3.작성일자: 2018. 12. 27. 21:55
4.블로그내용: 
   안녕하세요.제주항공 조이버 15기 홍소영입니다.오늘은 겨울인 대한민국을 떠나 더운 국가로 여행을 가는 분들을 위한제주항공의 꿀서비스를 소개해드리고자 해요.  바로,외투보관, 수하물보관/배송 서비스입니다!   다들 여름 여행지로 여행을 갈 때, 고민하는 게 무거운 겨울 외투를 어떻게 해야할 지 잖아요!그 해결을 제주항공이 도와드립니다.외투보관서비스는외투보관 예약하기 버튼을 통해 홈페이지로 들어가할인코드(jejuair)을 입력하고 예약하면 돼요.그리고 현장에서 외투를 보관하고 보관증을 수령한 뒤,여행이 끝나고 찾아가면 된답니다.원래 7일에 14,000원인데 제주항공 할인가로 9,000원에 이용할 수 있어요.   제주항공은 사이판 여행객을 위해 할인에 할인을 또! 해줍니다.위에 언급한 할인가에서 2,000원을 또 할인해7,000원에 이용할 수 있어요!진짜 저렴하죠ㅠㅠㅠ   -위치 및 영업시간-제주항공 외투보관/수하물보관서비스는인천공항 1터미널 트래블스토어, 인천공항 2터미널 트래블스토어, 김포공항 트래블센터에서 이용할 수 있어요!​   수하물 보관뿐만 아니라 배송도 가능하니 참고하세요! 메인 | 제주항공일반결제 포인트결제 왕복 편도 다구간 단체예약 신청 출발일선택 도착일선택 나이계산기 예매하기 리프레시 포인트 항공 마일리지의 새로운 기준! 항공운임 기준 5%적립 탑승권 이영역은 제주항공 메인화면의 광고영역으로 실시간으로 변화되는 콘텐츠가 존재합니다. 시작버튼과 멈춤버튼을 이용하여 실시간 콘텐츠를 제어 할 수 있습니다. 메인 1 번째 배너 메인 2 번째 배너 메인 3 번째 배너 메인 4 번째 배너 메인 5 번째 배너 2019년 여행결심 하면돼지 12월~3월 국내

총 17 건 중 11 번째 블로그 데이터를 수집합니다==============
1.블로그주소:  https://blog.naver.com/gimg0629?Redirect=Log&logNo=221424215543
2.작성자 닉네임:  GYoni
3.작성일자: 2018. 12. 21. 15:30
4.블로그내용: 
 ​    친구랑 당일치기 국내여행 ​​사진첩을 정리하다보니 추억을 기록해두지않은사진이 생각보다 많이 있어서하나씩 하나씩 올리는 중이예요.-이번엔 친구랑 당일치기 뚜벅이여행을 갔던 여수편입니다.친구랑은 어렸을때부터 친하게 지냈는데같이 여행은 몇번못간것같아요.같이가려고하면 이상하게 스케줄이 계속 꼬이더라구요.( ㅠㅠ )​​   ​​부산에서 여수로 GOGO !!! ​도착하자마자 우리가 간곳은 여수 간장게장골목이예요.보통 여수는 밤바다라고 하지만 우리는 여수 = 간장게장우리는 여름에 갔었는데 너무 더워서힘들고 기분이 저기업으로 뚝뚝 떨어지기에주문하면서 기원한 보리음료도 시켜보았아요.-반주로 각 한병씩먹으니 기분이 살 - 좋아지더라구요히히힛​​   금강산도 식후경 ​​​여수 게장골목중에 눈에 딱 보이는곳에들어갔는데 전에 왔언곳이라더라구요 게장이 무한리필이였어요.+_+게장은 맘먹고 먹으면 밥을 세공기 네공기 뚝딱여수에 여행가는 분들은 대부분점심 , 저녁은 간장게장으로 메뉴를 고르는것같아요.전에 왔을때도 점심을 게장으로 먹었었는데예전 기억이 스물스물 나더라구요.​ ​​   상다리 부러지도록 준비되었어요. ​전체적으로 짭쪼롬한 반찬들로밥을 계속 먹게된답니다.-친구는 게장을 거의 안먹어봤다고처음엔 별로 안좋아한다고헀는데나중에는 저보다 더 맛있게 먹고있더라구요첨엔 매콤한 양념이 맛있다고하더니 마지막엔 간장게장이 진리라고하던 내친구가 기억이나네요( 후후훗 )​​   양념게장   간장게장 ​  밥을 먹고 이동한곳은 여수해양레일바이크뚜벅이여행을 가시는분들은레일바이크를 타기위해선 터널을 지나야 하기때문에택시를 타고 들어가야해요.-입구에서 티켓을 구매하고 들어가면 바로탈수있어요다